Мы делаем CatBoost повторно, поэтому сочтём, что здесь будет меньше наших комментариев, так как этот файл является почти полной копией файла TrainDataCatBoost2.ipynb

И мы позволим себе просто использовать более расширенный датасет, в который теперь добавился год выпуска браузера и год выпуска операционной системы.

## Повторение CatBoost

In [1]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

Теперь можно открыть файлы, которые мы подготовили ранее

In [2]:
train_df_csv = pd.read_csv('../data/train_processed_with_years.csv',delimiter = ';')
test_df_csv = pd.read_csv('../data/test_processed_with_years.csv',delimiter = ';')

In [3]:
train_df_csv

,request_ts,user_id,country_id,region_id,timezone_id,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,browser,os,target,browser_years_old,os_years_old
0,1701011363,fb858e8e0a2bec074450eaf94b627fd3,c31b4e,470e75,f6155e,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,Android,0,1.0,9.714804
1,1700986581,46a5f128fd569c764a92c2eaa788095e,c31b4e,44520b,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,Android,0,1.0,9.714804
2,1701011071,5a74e9ac53ffb21a20cce117c0ad77ba,c31b4e,616bb9,af47f1,12498,2451,10304,-6380,11608,3106,-2188,10573,3347,21870,Yandex Browser,Android,0,4.0,9.714804
3,1700992803,af735816ca19115431ae3d89518c8c91,c31b4e,3c9dca,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,Android,0,1.0,9.714804
4,1701021666,364f0ae0a3f29a685c4fb5bae6033b9a,c31b4e,776e76,10b7947,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Yandex Browser,Android,0,7.0,9.714804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601285,1701009148,11516096d9f97463424523e6154d5255,c31b4e,8ccc01,e56e80,18174,17591,3257,2581,8043,13263,2312,1791,9121,5170,Yandex Browser,Android,1,2.0,9.714804
601286,1700954940,d078ecb7c7e9eaf65189fb3d1bed7871,c31b4e,52b73e,e56e80,14961,13188,3978,1812,13751,14055,1141,-10698,7083,9270,Chrome,Windows,0,1.0,9.000000
601287,1700994910,3543d64627ead3a519e3199834e2a148,c31b4e,33ed7a,10b7947,12332,17481,12664,-984,5146,11562,4316,15017,1423,-981,Chrome Mobile,Android,0,2.0,9.714804
601288,1700989030,37df5ff1d739f61d442b164db6281e46,c31b4e,97023a,e56e80,13459,-6413,13160,13187,5498,3875,13246,10259,-1630,-14647,Chrome Mobile,Android,0,3.0,9.714804


In [4]:
X = train_df_csv.drop(columns=['target', 'user_id'])
y = train_df_csv['target']

Разделим данные на обучающущую и тестовую выборку

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

Отдельно отметим наличие категориальных признаков, это country_id, region_id, timezone_id, os, browser.

In [6]:
cat_features = ['country_id', 'region_id', 'timezone_id', 'os', 'browser']

Как выяснилось библиотека не приемлет наличие всяких около наловых значений в категориальных признаках, так что уберём всякие NaN, None и прочее. Теперь это отдельная категория - Other.

In [7]:
X_train.isnull()

,request_ts,country_id,region_id,timezone_id,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,browser,os,browser_years_old,os_years_old
385201,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
483415,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
551633,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
328803,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
265436,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
259178,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
365838,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
131932,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [8]:
X_train[cat_features] = X_train[cat_features].replace({None: 'Other', '': 'Other'})
X_train[cat_features] = X_train[cat_features].fillna('Other')

Создаём пул

In [9]:
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
#test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)

Теперь инициализируем модель. Параметры взял из примера, возможно, они нам не понравятся и мы пойдём их менять далее.

In [10]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=100)

Теперь происходит обучение

In [11]:
model.fit(train_pool)

0:	learn: 0.6671306	total: 979ms	remaining: 16m 18s
100:	learn: 0.4828887	total: 52.6s	remaining: 7m 48s
200:	learn: 0.4616254	total: 1m 44s	remaining: 6m 55s
300:	learn: 0.4510281	total: 2m 35s	remaining: 6m 1s
400:	learn: 0.4441743	total: 3m 26s	remaining: 5m 8s
500:	learn: 0.4390482	total: 4m 17s	remaining: 4m 16s
600:	learn: 0.4351039	total: 5m 10s	remaining: 3m 26s
700:	learn: 0.4319484	total: 6m 9s	remaining: 2m 37s
800:	learn: 0.4290921	total: 7m	remaining: 1m 44s
900:	learn: 0.4265266	total: 7m 54s	remaining: 52.1s
999:	learn: 0.4241465	total: 8m 53s	remaining: 0us


In [12]:
X_test[cat_features] = X_test[cat_features].replace({None: 'Other', '': 'Other'})
X_test[cat_features] = X_test[cat_features].fillna('Other')

Проверим точность

In [13]:
accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.82


In [14]:
from sklearn.metrics import f1_score 

def GetPrediction(model, X, y, metric = f1_score, **kwargs):
    
    prediction = model.predict(X)
    return metric(y, prediction, **kwargs)

In [15]:
f" f1 на тренировочной выборке: {GetPrediction(model, X_train, y_train):.2f}" +'\n'+f" f1 на валидационной выборке: {GetPrediction(model, X_test, y_test):.2f}"

' f1 на тренировочной выборке: 0.82\n f1 на валидационной выборке: 0.81'

С последней нашей попытки мы получили такие 0.81, впрочем наши последние попытки не увенчались успехом, они были для нас ценным опытом.

## Применение и сохранение результата


Сейчас нам необходимо сформировать предсказание согласно техническому заданию для тестового файла.

In [17]:
test_df_csv

,request_ts,user_id,country_id,region_id,timezone_id,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,browser,os,browser_years_old,os_years_old
0,1700993094,c2802dadd33d8ae09bb366bdd41212ea,c31b4e,36e3f3,f6155e,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,Android,1.0,9.714804
1,1701005579,e5b1988db74527ec092f28b0bbfdaac9,c31b4e,8ccc01,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome,Android,1.0,9.714804
2,1700969752,6ef1eedbdb72554e53e69782066065c5,c31b4e,1fbfa5,e56e80,-7307,11682,9741,13564,13577,1200,10169,16461,-3932,3340,Chrome,Android,4.0,9.714804
3,1700991608,7e057293ecae62985a327b7af51858ea,c31b4e,f66ff,f6155e,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,Android,1.0,9.714804
4,1701019815,a27bd7ce8828497823fa8d5d05e7bbf7,c31b4e,245864,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,Android,7.0,9.714804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153078,1700968705,3c86b41e45cc2b7029d969e4e045fc8d,c31b4e,5f9c68,9cdb77,-7489,3356,11800,-9387,20562,10689,13210,-2615,9753,1102,Chrome,Windows,2.0,9.714804
153079,1700989358,8daa3e87a079f857f559062fbf2c02e1,c31b4e,776e76,10b7947,8011,11687,1439,13197,5889,-3233,14858,5281,14728,11103,Yandex Browser,Android,2.0,9.714804
153080,1701025826,8bd30a0a06100a0c782046ec151fcf03,c31b4e,1b18a1,e56e80,10214,-1254,10495,7705,9547,5801,864,8837,5312,22635,YandexSearch,Android,2.0,9.714804
153081,1700979975,0bba77ee4a9a5f03c7dac143b77a7a0d,c31b4e,8ccc01,e56e80,2794,-10182,-18072,-4222,18836,-10290,7087,-2586,17127,-921,Chrome,Windows,2.0,9.714804


Пока отложим user_id в отдельный массив, чтобы данные были максимально похожими на то, на чём модель училась.

In [24]:
input_data = test_df_csv.drop(columns=['user_id'])
user_ids =test_df_csv['user_id']

input_data

,request_ts,country_id,region_id,timezone_id,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,browser,os,browser_years_old,os_years_old
0,1700993094,c31b4e,36e3f3,f6155e,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,Android,1.0,9.714804
1,1701005579,c31b4e,8ccc01,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome,Android,1.0,9.714804
2,1700969752,c31b4e,1fbfa5,e56e80,-7307,11682,9741,13564,13577,1200,10169,16461,-3932,3340,Chrome,Android,4.0,9.714804
3,1700991608,c31b4e,f66ff,f6155e,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,Android,1.0,9.714804
4,1701019815,c31b4e,245864,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,Android,7.0,9.714804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153078,1700968705,c31b4e,5f9c68,9cdb77,-7489,3356,11800,-9387,20562,10689,13210,-2615,9753,1102,Chrome,Windows,2.0,9.714804
153079,1700989358,c31b4e,776e76,10b7947,8011,11687,1439,13197,5889,-3233,14858,5281,14728,11103,Yandex Browser,Android,2.0,9.714804
153080,1701025826,c31b4e,1b18a1,e56e80,10214,-1254,10495,7705,9547,5801,864,8837,5312,22635,YandexSearch,Android,2.0,9.714804
153081,1700979975,c31b4e,8ccc01,e56e80,2794,-10182,-18072,-4222,18836,-10290,7087,-2586,17127,-921,Chrome,Windows,2.0,9.714804


Заменим всякое нехорошее общей категорией - Other

In [25]:

input_data[cat_features] = input_data[cat_features].replace({None: 'Other', '': 'Other'})
input_data[cat_features] = input_data[cat_features].fillna('Other')

Позволим нашей модели сделать предсказание

In [26]:
prediction = model.predict(input_data)

prediction

array([0, 0, 1, ..., 1, 1, 1], dtype=int64)

Преобразуем всё полученное в ДатаФреймы, склеиваем их. Получается нечто похожее на train_labels.csv

In [32]:
prediction = pd.DataFrame(prediction)
user_ids = pd.DataFrame(user_ids)

user_ids[['target']] = prediction.apply(pd.Series)

user_ids

,user_id,target
0,c2802dadd33d8ae09bb366bdd41212ea,0
1,e5b1988db74527ec092f28b0bbfdaac9,0
2,6ef1eedbdb72554e53e69782066065c5,1
3,7e057293ecae62985a327b7af51858ea,0
4,a27bd7ce8828497823fa8d5d05e7bbf7,0
...,...,...
153078,3c86b41e45cc2b7029d969e4e045fc8d,1
153079,8daa3e87a079f857f559062fbf2c02e1,0
153080,8bd30a0a06100a0c782046ec151fcf03,1
153081,0bba77ee4a9a5f03c7dac143b77a7a0d,1


И финальный штрих в нашей работе. Сохраняем полученные финальные предсказания в файл, который по аналогии видимо стоит назвать test_labels.csv

In [33]:
user_ids.to_csv('../data/test_labels.csv', sep=';', index=False)